In [1]:
import dask.dataframe as pd

d_diagnoses_df = pd.read_csv("hosp/d_icd_diagnoses.csv",dtype={'icd_code': 'object'})
patients_df = pd.read_csv("hosp/patients.csv")
lab_events_df = pd.read_csv("hosp/labevents.csv")

In [2]:
lab_events_df["hadm_id"].nunique().compute()

351034

In [3]:
hemato_events = lab_events_df[lab_events_df["itemid"].isin([  51301, # White Blood Cells, K/uL, Blood, Hematology, LOINC 804-5
            51279, # Red Blood Cells, m/uL, Blood, Hematology, LOINC 789-8
            51265, # Platelet Count, K/uL, Blood, Hematology, LOINC 777-3
            51222, # Hemoglobin, g/dL, Blood, Hematology, LOINC 718-7,
            51250  # MCV, fL, Blood, Hematology, LOINC 787-2
])]
hemato_events = hemato_events.dropna(subset="valuenum")
hemato_events = hemato_events.drop_duplicates(subset=["subject_id", "hadm_id", "charttime", "itemid"])

In [4]:
hemato_df = hemato_events.compute() ## dask doesnt allow multi index (needed for pivot table creation)

/home/dwalke/.local/lib/python3.10/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/home/dwalke/.local/lib/python3.10/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (5,8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [5]:
hemato_df["hadm_id"].nunique()

332080

### Some labevents do not have an hadmission id; we need the hadm id to merge with the diagnoses
### However, we also want to include labevents without hadmission id which will be considered additionally as controls

In [6]:
pivot_hemato_df_wo_hadm = hemato_df.pivot_table(index=["subject_id",  "charttime"],columns='itemid', values="valuenum", aggfunc='first').reset_index()
hemato_df_pivot = pd.merge(pivot_hemato_df_wo_hadm, hemato_df[["subject_id", "charttime", "hadm_id"]].drop_duplicates(), on=["subject_id", "charttime"], how="left")

In [7]:
# pivot_hemato_df_wo_hadm = hemato_df.pivot_table(index=["subject_id",  "charttime"],columns='itemid', values="valuenum", aggfunc='first').reset_index()
# pivot_hemato_df_w_hadm = hemato_df.pivot_table(index=["subject_id",  "charttime", "hadm_id"],columns='itemid', values="valuenum", aggfunc='first').reset_index()
# hemato_df_pivot =  pd.merge(left=pivot_hemato_df_wo_hadm, right=pivot_hemato_df_w_hadm, how="left", on=["subject_id", "charttime"], suffixes=("", "_y"))

### we will join both dfs with left join to keep all labevents 

In [8]:
hemato_df_pivot.astype({"hadm_id": "category", "subject_id": "category", "charttime": "datetime64[ns]"})
hemato_df_pivot

,subject_id,charttime,51222,51250,51265,51279,51301,hadm_id
0,10000032,2180-03-23 11:51:00,14.9,103.0,83.0,4.40,3.0,NaN
1,10000032,2180-05-06 22:25:00,14.3,99.0,71.0,4.29,5.0,NaN
2,10000032,2180-05-07 05:05:00,12.7,99.0,71.0,3.80,4.2,22595853.0
3,10000032,2180-06-22 11:15:00,14.0,101.0,113.0,4.14,5.1,NaN
4,10000032,2180-06-26 16:10:00,13.4,100.0,143.0,3.84,6.6,NaN
...,...,...,...,...,...,...,...,...
3230905,19999987,2145-11-04 10:40:00,12.6,105.0,120.0,3.64,11.6,23865745.0
3230906,19999987,2145-11-05 06:10:00,12.3,105.0,135.0,3.58,10.0,23865745.0
3230907,19999987,2145-11-06 10:07:00,12.7,106.0,141.0,3.63,5.9,23865745.0
3230908,19999987,2145-11-07 06:00:00,11.5,106.0,129.0,3.32,5.0,23865745.0


In [9]:
pivot_hemato_df = hemato_df_pivot.rename(columns={51222: "HGB", 51250: "MCV", 51265: "PLT", 51279:"RBC", 51301: "WBC"})
pivot_hemato_df.loc[:, "HGB"] = pivot_hemato_df.loc[:, "HGB"]*0.621

In [10]:
patients_df = patients_df.compute()

In [11]:
import pandas as pd
import numpy as np
transfer_df = pd.read_csv("hosp/transfers.csv")
icu_map_df = pd.read_csv("icu_map.csv")

In [12]:
transfer_df = transfer_df[transfer_df.loc[:, "eventtype"] !=  "discharge"]
transfer_df['intime']= pd.to_datetime(transfer_df['intime'])
transfer_df['outtime']= pd.to_datetime(transfer_df['outtime'])
transfer_joined = pd.merge(left=transfer_df, right=icu_map_df, left_on="careunit", right_on="Icu", how="left")
# create and merge fake lab events
icu_transfers = transfer_joined[transfer_joined["Type"].str.contains("ICU") == True] #some types equals NaN resulting in NaN with drect comparsion -> therefore additional compairson with True
icu_transfers = icu_transfers.assign(charttime=lambda df: df["intime"] + (df["outtime"] - df["intime"]) / 2)
icu_transfers = icu_transfers.assign(HGB=lambda df: -np.inf)
icu_transfers = icu_transfers.loc[:, ["subject_id", "charttime", "HGB"]]
labevents = pd.concat([pivot_hemato_df, icu_transfers], axis=0, ignore_index=True, sort=False)

In [13]:
labevents.shape

(3313010, 8)

In [14]:
labevents["charttime"] = pd.to_datetime(labevents["charttime"])
transfer_joined = transfer_joined.assign(jintime=lambda df: df["intime"])
transfer_joined = transfer_joined.assign(jouttime=lambda df: df["outtime"])
labevents = labevents.assign(jcharttime=lambda df: df["charttime"])

In [15]:
labevents.shape, transfer_joined.shape

((3313010, 9), (1459741, 11))

In [16]:
transfer_hemato_join = pd.merge(labevents, transfer_joined, on="subject_id", how="left", suffixes=("", "_y"))
transfer_hemato_join.shape

(67355031, 19)

In [17]:
#transfer_hemato_join.pop("hadm_id_y") #

In [18]:
transfer_hemato_join_red = transfer_hemato_join[(transfer_hemato_join["jcharttime"] <  transfer_hemato_join["jouttime"])&(transfer_hemato_join["jcharttime"] >=  transfer_hemato_join["jintime"])]

In [19]:
# the left join with additional filter is not equal to an non equi join, i.e., labevents that couldnt be joined with a transfer were filtered out
transfer_hemato_join_red= transfer_hemato_join_red.assign(subject_id_charttime=lambda x: x["subject_id"].astype("str") + x["charttime"].astype("str"))

In [20]:
labevents = labevents.assign(subject_id_charttime=lambda x: x["subject_id"].astype("str") + x["charttime"].astype("str"))

In [21]:
subject_id_charttimes_strings = transfer_hemato_join_red["subject_id_charttime"].values

In [22]:
transfer_hemato_join_red.shape

(2301980, 20)

In [23]:
missing_data = labevents[~(labevents["subject_id_charttime"].isin(subject_id_charttimes_strings))]#.shape[0] +transfer_hemato_join_red.shape[0]

In [24]:
missing_data["hadm_id"]#.nunique()

0                 NaN
3                 NaN
12                NaN
21                NaN
23                NaN
              ...    
3230853           NaN
3230854           NaN
3230855           NaN
3230856           NaN
3230890    26071774.0
Name: hadm_id, Length: 1011108, dtype: float64

In [25]:
transfer_hemato_join_red

,subject_id,charttime,HGB,MCV,PLT,RBC,WBC,hadm_id,jcharttime,hadm_id_y,transfer_id,eventtype,careunit,intime,outtime,Icu,Type,jintime,jouttime,subject_id_charttime
11,10000032,2180-05-06 22:25:00.000,8.8803,99.0,71.0,4.29,5.0,NaN,2180-05-06 22:25:00.000,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00,Emergency Department,ED,2180-05-06 19:17:00,2180-05-06 23:30:00,100000322180-05-06 22:25:00.000
23,10000032,2180-05-07 05:05:00.000,7.8867,99.0,71.0,3.80,4.2,22595853.0,2180-05-07 05:05:00.000,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27,NaN,NaN,2180-05-06 23:30:00,2180-05-07 17:21:27,100000322180-05-07 05:05:00.000
47,10000032,2180-06-26 16:10:00.000,8.3214,100.0,143.0,3.84,6.6,NaN,2180-06-26 16:10:00.000,22841357.0,38112554,ED,Emergency Department,2180-06-26 15:54:00,2180-06-26 21:31:00,Emergency Department,ED,2180-06-26 15:54:00,2180-06-26 21:31:00,100000322180-06-26 16:10:00.000
57,10000032,2180-06-27 05:10:00.000,7.7004,99.0,137.0,3.60,6.6,22841357.0,2180-06-27 05:10:00.000,22841357.0,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12,NaN,NaN,2180-06-26 21:31:00,2180-06-27 18:49:12,100000322180-06-27 05:10:00.000
75,10000032,2180-07-23 06:39:00.000,8.7561,103.0,145.0,3.98,6.9,NaN,2180-07-23 06:39:00.000,29079034.0,39399961,ED,Emergency Department,2180-07-23 05:54:00,2180-07-23 14:00:00,Emergency Department,ED,2180-07-23 05:54:00,2180-07-23 14:00:00,100000322180-07-23 06:39:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67355005,19999625,2139-10-11 06:49:44.000,-inf,NaN,NaN,NaN,NaN,NaN,2139-10-11 06:49:44.000,25304202.0,31070865,admit,Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,Medical/Surgical Intensive Care Unit (MICU/SICU),CICU,2139-10-10 19:18:00,2139-10-11 18:21:28,199996252139-10-11 06:49:44.000
67355013,19999828,2149-01-09 15:41:31.000,-inf,NaN,NaN,NaN,NaN,NaN,2149-01-09 15:41:31.000,25744818.0,36075953,admit,Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,Medical Intensive Care Unit (MICU),MICU,2149-01-08 18:12:00,2149-01-10 13:11:02,199998282149-01-09 15:41:31.000
67355018,19999840,2164-09-15 13:06:31.500,-inf,NaN,NaN,NaN,NaN,NaN,2164-09-15 13:06:31.500,21033226.0,34767778,transfer,Surgical Intensive Care Unit (SICU),2164-09-13 09:37:48,2164-09-17 16:35:15,Surgical Intensive Care Unit (SICU),SICU,2164-09-13 09:37:48,2164-09-17 16:35:15,199998402164-09-15 13:06:31.500
67355025,19999840,2164-09-12 21:32:08.000,-inf,NaN,NaN,NaN,NaN,NaN,2164-09-12 21:32:08.000,21033226.0,38978960,transfer,Trauma SICU (TSICU),2164-09-12 09:26:28,2164-09-13 09:37:48,Trauma SICU (TSICU),SICU,2164-09-12 09:26:28,2164-09-13 09:37:48,199998402164-09-12 21:32:08.000


In [26]:
missing_data ##TODO join ths with transfer_hemato_join_red and have finally the non-equi join

,subject_id,charttime,HGB,MCV,PLT,RBC,WBC,hadm_id,jcharttime,subject_id_charttime
0,10000032,2180-03-23 11:51:00,9.2529,103.0,83.0,4.40,3.0,NaN,2180-03-23 11:51:00,100000322180-03-23 11:51:00.000
3,10000032,2180-06-22 11:15:00,8.6940,101.0,113.0,4.14,5.1,NaN,2180-06-22 11:15:00,100000322180-06-22 11:15:00.000
12,10000032,2180-08-10 12:00:00,7.2036,103.0,136.0,3.27,8.0,NaN,2180-08-10 12:00:00,100000322180-08-10 12:00:00.000
21,10000117,2174-06-03 09:00:00,9.3150,93.0,329.0,4.91,6.6,NaN,2174-06-03 09:00:00,100001172174-06-03 09:00:00.000
23,10000117,2176-02-21 10:12:00,9.2529,94.0,268.0,4.71,8.4,NaN,2176-02-21 10:12:00,100001172176-02-21 10:12:00.000
...,...,...,...,...,...,...,...,...,...,...
3230853,19999784,2122-01-23 08:05:00,8.1972,85.0,260.0,4.86,3.3,NaN,2122-01-23 08:05:00,199997842122-01-23 08:05:00.000
3230854,19999784,2122-02-01 09:45:00,8.5077,85.0,256.0,4.94,3.3,NaN,2122-02-01 09:45:00,199997842122-02-01 09:45:00.000
3230855,19999784,2122-05-04 15:00:00,7.8867,85.0,252.0,4.55,4.3,NaN,2122-05-04 15:00:00,199997842122-05-04 15:00:00.000
3230856,19999784,2122-05-10 11:30:00,8.6319,85.0,269.0,4.95,3.0,NaN,2122-05-10 11:30:00,199997842122-05-10 11:30:00.000


In [27]:
labevents = pd.concat((transfer_hemato_join_red, missing_data))

In [28]:
## TODO Add this later because otherwise we would miss the hadm_ids from unjoined labevents
# transfer_hemato_join_red = transfer_hemato_join_red.rename(columns={"hadm_id_y": "hadm_id"})
# labevents = pd.concat((transfer_hemato_join_red, missing_data)) #389090 unique hadm_ids

## TODO Remove this later because otherwise we would miss the hadm_ids from unjoined labevents
labevents.pop("hadm_id")
labevents = labevents.rename(columns={"hadm_id_y": "hadm_id"}) #389012 unique hadm_ids

In [29]:
labevents["hadm_id"].nunique()

389012

In [30]:
labevents.loc[labevents["Type"].isna(), "Type"] = "GEN"

In [31]:
labevents

,subject_id,charttime,HGB,MCV,PLT,RBC,WBC,jcharttime,hadm_id,transfer_id,eventtype,careunit,intime,outtime,Icu,Type,jintime,jouttime,subject_id_charttime
11,10000032,2180-05-06 22:25:00,8.8803,99.0,71.0,4.29,5.0,2180-05-06 22:25:00,22595853.0,33258284.0,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00,Emergency Department,ED,2180-05-06 19:17:00,2180-05-06 23:30:00,100000322180-05-06 22:25:00.000
23,10000032,2180-05-07 05:05:00,7.8867,99.0,71.0,3.80,4.2,2180-05-07 05:05:00,22595853.0,35223874.0,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27,NaN,GEN,2180-05-06 23:30:00,2180-05-07 17:21:27,100000322180-05-07 05:05:00.000
47,10000032,2180-06-26 16:10:00,8.3214,100.0,143.0,3.84,6.6,2180-06-26 16:10:00,22841357.0,38112554.0,ED,Emergency Department,2180-06-26 15:54:00,2180-06-26 21:31:00,Emergency Department,ED,2180-06-26 15:54:00,2180-06-26 21:31:00,100000322180-06-26 16:10:00.000
57,10000032,2180-06-27 05:10:00,7.7004,99.0,137.0,3.60,6.6,2180-06-27 05:10:00,22841357.0,34703856.0,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12,NaN,GEN,2180-06-26 21:31:00,2180-06-27 18:49:12,100000322180-06-27 05:10:00.000
75,10000032,2180-07-23 06:39:00,8.7561,103.0,145.0,3.98,6.9,2180-07-23 06:39:00,29079034.0,39399961.0,ED,Emergency Department,2180-07-23 05:54:00,2180-07-23 14:00:00,Emergency Department,ED,2180-07-23 05:54:00,2180-07-23 14:00:00,100000322180-07-23 06:39:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3230853,19999784,2122-01-23 08:05:00,8.1972,85.0,260.0,4.86,3.3,2122-01-23 08:05:00,NaN,NaN,NaN,NaN,NaT,NaT,NaN,GEN,NaT,NaT,199997842122-01-23 08:05:00.000
3230854,19999784,2122-02-01 09:45:00,8.5077,85.0,256.0,4.94,3.3,2122-02-01 09:45:00,NaN,NaN,NaN,NaN,NaT,NaT,NaN,GEN,NaT,NaT,199997842122-02-01 09:45:00.000
3230855,19999784,2122-05-04 15:00:00,7.8867,85.0,252.0,4.55,4.3,2122-05-04 15:00:00,NaN,NaN,NaN,NaN,NaT,NaT,NaN,GEN,NaT,NaT,199997842122-05-04 15:00:00.000
3230856,19999784,2122-05-10 11:30:00,8.6319,85.0,269.0,4.95,3.0,2122-05-10 11:30:00,NaN,NaN,NaN,NaN,NaT,NaT,NaN,GEN,NaT,NaT,199997842122-05-10 11:30:00.000


In [32]:
## only consider binary gender for simplicity aacording to Ahrens et al.
patients_df["Sex"] = patients_df["gender"].apply(lambda x: "W" if x == "F" else "M")

In [33]:
patients_df

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,Sex
0,10000032,F,52,2180,2014 - 2016,2180-09-09,W
1,10000048,F,23,2126,2008 - 2010,NaN,W
2,10000068,F,19,2160,2008 - 2010,NaN,W
3,10000084,M,72,2160,2017 - 2019,2161-02-13,M
4,10000102,F,27,2136,2008 - 2010,NaN,W
...,...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,NaN,W
299708,19999829,F,28,2186,2008 - 2010,NaN,W
299709,19999840,M,58,2164,2008 - 2010,2164-09-17,M
299710,19999914,F,49,2158,2017 - 2019,NaN,W


In [34]:
patients_df = patients_df.assign(Birth=lambda df: df["anchor_year"] - df["anchor_age"])

In [35]:
lab_patients_events = pd.merge(labevents, patients_df.loc[:, ("gender", "Birth", "subject_id")], on ="subject_id", how="left")

In [36]:
lab_patients_events = lab_patients_events.assign(Age=lambda df: df["charttime"].dt.year - df["Birth"])

In [37]:
lab_patients_events.pop("Birth")

0          2128
1          2128
2          2128
3          2128
4          2128
           ... 
3313083    2062
3313084    2062
3313085    2062
3313086    2062
3313087    2106
Name: Birth, Length: 3313088, dtype: int64

In [38]:
lab_patients_events = lab_patients_events[lab_patients_events["Age"] >= 18]

In [39]:
lab_patients_events["hadm_id"].nunique()

389012

In [40]:
lab_patients_events

,subject_id,charttime,HGB,MCV,PLT,RBC,WBC,jcharttime,hadm_id,transfer_id,...,careunit,intime,outtime,Icu,Type,jintime,jouttime,subject_id_charttime,gender,Age
0,10000032,2180-05-06 22:25:00,8.8803,99.0,71.0,4.29,5.0,2180-05-06 22:25:00,22595853.0,33258284.0,...,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00,Emergency Department,ED,2180-05-06 19:17:00,2180-05-06 23:30:00,100000322180-05-06 22:25:00.000,F,52
1,10000032,2180-05-07 05:05:00,7.8867,99.0,71.0,3.80,4.2,2180-05-07 05:05:00,22595853.0,35223874.0,...,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27,NaN,GEN,2180-05-06 23:30:00,2180-05-07 17:21:27,100000322180-05-07 05:05:00.000,F,52
2,10000032,2180-06-26 16:10:00,8.3214,100.0,143.0,3.84,6.6,2180-06-26 16:10:00,22841357.0,38112554.0,...,Emergency Department,2180-06-26 15:54:00,2180-06-26 21:31:00,Emergency Department,ED,2180-06-26 15:54:00,2180-06-26 21:31:00,100000322180-06-26 16:10:00.000,F,52
3,10000032,2180-06-27 05:10:00,7.7004,99.0,137.0,3.60,6.6,2180-06-27 05:10:00,22841357.0,34703856.0,...,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12,NaN,GEN,2180-06-26 21:31:00,2180-06-27 18:49:12,100000322180-06-27 05:10:00.000,F,52
4,10000032,2180-07-23 06:39:00,8.7561,103.0,145.0,3.98,6.9,2180-07-23 06:39:00,29079034.0,39399961.0,...,Emergency Department,2180-07-23 05:54:00,2180-07-23 14:00:00,Emergency Department,ED,2180-07-23 05:54:00,2180-07-23 14:00:00,100000322180-07-23 06:39:00.000,F,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313083,19999784,2122-01-23 08:05:00,8.1972,85.0,260.0,4.86,3.3,2122-01-23 08:05:00,NaN,NaN,...,NaN,NaT,NaT,NaN,GEN,NaT,NaT,199997842122-01-23 08:05:00.000,M,60
3313084,19999784,2122-02-01 09:45:00,8.5077,85.0,256.0,4.94,3.3,2122-02-01 09:45:00,NaN,NaN,...,NaN,NaT,NaT,NaN,GEN,NaT,NaT,199997842122-02-01 09:45:00.000,M,60
3313085,19999784,2122-05-04 15:00:00,7.8867,85.0,252.0,4.55,4.3,2122-05-04 15:00:00,NaN,NaN,...,NaN,NaT,NaT,NaN,GEN,NaT,NaT,199997842122-05-04 15:00:00.000,M,60
3313086,19999784,2122-05-10 11:30:00,8.6319,85.0,269.0,4.95,3.0,2122-05-10 11:30:00,NaN,NaN,...,NaN,NaT,NaT,NaN,GEN,NaT,NaT,199997842122-05-10 11:30:00.000,M,60


## Reading diagnoses

In [41]:
diagnoses_df = pd.read_csv("hosp/diagnoses_icd.csv")

In [42]:
diagnoses_df["SIRS"] = diagnoses_df["icd_code"].str.contains("99590|R651[01]")

In [43]:
diagnoses_df["Sepsis"] = diagnoses_df["icd_code"].isin([ "78552", "99591", "99592",
  "A021", "A207", "A227", "A241", "A267", "A327", "A392", "A393", "A394",
  "A40", "A400", "A401", "A403", "A408", "A409", "A41", "A410", "A4101",
  "A4102", "A411", "A412", "A413", "A414", "A4150", "A4151", "A4152",
  "A4153", "A4159", "A418", "A4181", "A4189", "A419", "A427", "B377",
  "R6520", "R6521"])

In [44]:
sirs_sepsis = diagnoses_df.groupby(["subject_id", "hadm_id"]).agg({"SIRS": "max", "Sepsis": "max"}).reset_index()

In [45]:
sirs_sepsis["Diagnosis"] = sirs_sepsis.apply(lambda row: ["Control", "SIRS", "Sepsis"][row["SIRS"] + (2 - row["SIRS"]) * row["Sepsis"]], axis=1)

In [46]:
sirs_sepsis.pop("SIRS")
sirs_sepsis.pop("Sepsis")

0         False
1         False
2         False
3         False
4         False
          ...  
430847    False
430848    False
430849    False
430850    False
430851    False
Name: Sepsis, Length: 430852, dtype: bool

In [47]:
lab_patients_events.loc[:, ["subject_id", "hadm_id"]].nunique()

subject_id    242843
hadm_id       389012
dtype: int64

In [48]:
lab_patients_diag_events = pd.merge(lab_patients_events, sirs_sepsis, on=["subject_id", "hadm_id"], how="left")

In [49]:
lab_patients_diag_events.loc[lab_patients_diag_events["Diagnosis"].isna(), "Diagnosis"] = "Control"

In [50]:
lab_patients_diag_events.loc[lab_patients_diag_events["transfer_id"].isna(), "transfer_id"] = 0

In [51]:
print((lab_patients_diag_events["Diagnosis"] == "Control").sum())
print((lab_patients_diag_events["Diagnosis"] == "Sepsis").sum())
print((lab_patients_diag_events["Diagnosis"] == "SIRS").sum())

3039265
270168
3560


In [52]:
lab_patients_diag_events.shape

(3312993, 22)

In [53]:
import pandas as pd
import numpy as np
# Create a sample dataframe
data = {
    'Category': ['A', 'A', 'B', 'B', 'C', 'C'],
    'Group': ['X', np.nan, 'X', 'Y', 'X', 'Y'],
    'Value': [1, 1, 3, 4, 5, 6]
}
df = pd.DataFrame(data)
print(df.shape)
df_grouped_reset = df.groupby(['Category', 'Group']).apply(lambda x: x).reset_index()
print("\nDataFrame after resetting the index:")
print(df_grouped_reset)


(6, 3)

DataFrame after resetting the index:
   index Category Group  Value
0      0        A     X      1
1      2        B     X      3
2      3        B     Y      4
3      4        C     X      5
4      5        C     Y      6


In [61]:
import dask.dataframe as dd

dd_lab_patients_diag_events = dd.from_pandas(lab_patients_diag_events,npartitions=1000)

In [70]:
dd_lab_patients_diag_events['subject_id'] = dd_lab_patients_diag_events['subject_id'].fillna('Unknown')
dd_lab_patients_diag_events['hadm_id'] = dd_lab_patients_diag_events['hadm_id'].fillna('Unknown')

In [ ]:
def add_new_ward(group):
    group = group.sort_values("charttime")
    first_transfer_id = group.loc[:, "transfer_id"].iloc[0]
    group["NewWard"] = group["transfer_id"].ne(first_transfer_id)
    return group


dd_lab_patients_diag_events.groupby(["subject_id", "hadm_id"]).apply(add_new_ward).reset_index(drop=True).compute().shape

/tmp/ipykernel_622714/1663328189.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_lab_patients_diag_events.groupby(["subject_id", "hadm_id"]).apply(add_new_ward).reset_index(drop=True).compute().shape


In [ ]:
def add_new_ward(group):
    group = group.sort_values("charttime")
    first_transfer_id = group.loc[:, "transfer_id"].iloc[0]
    group["NewWard"] = group["transfer_id"].ne(first_transfer_id)
    return group

lab_patients_diag_events['subject_id'].fillna('Unknown', inplace=True)
lab_patients_diag_events['hadm_id'].fillna('Unknown', inplace=True)
print(lab_patients_diag_events.shape)
lab_patients_diag_events_new_ward = lab_patients_diag_events.groupby(["subject_id", "hadm_id"]).apply(add_new_ward).reset_index(drop=True)
lab_patients_diag_events_new_ward.shape

(3312993, 24)


In [ ]:
lab_patients_diag_events_new_ward.head(20).loc[:,["hadm_id","subject_id", "charttime", "NewWard", "HGB", "MCV", "WBC", "PLT"]]

In [53]:
lab_patients_diag_events_new_ward = lab_patients_diag_events_new_ward.reset_index(drop=True)

In [65]:
lab_patients_diag_events_new_ward

subject_id           charttime     HGB    MCV  \
subject_id hadm_id                                                             
10000032   22595853.0 1          10000032 2180-05-07 05:05:00  7.8867   99.0   
           22841357.0 3          10000032 2180-06-27 05:10:00  7.7004   99.0   
           25742920.0 8          10000032 2180-08-06 06:36:00  7.5141  103.0   
                      9          10000032 2180-08-07 06:15:00  7.2036  103.0   
           29079034.0 5          10000032 2180-07-24 06:35:00  7.3899  102.0   
...                                   ...                 ...     ...    ...   
19999987   23865745.0 2219875    19999987 2145-11-04 10:40:00  7.8246  105.0   
                      2219876    19999987 2145-11-05 06:10:00  7.6383  105.0   
                      2219877    19999987 2145-11-06 10:07:00  7.8867  106.0   
                      2219878    19999987 2145-11-07 06:00:00  7.1415  106.0   
                      2219879    19999987 2145-11-09 05:30:00  7.6383  104.0   

                                 PLT   RBC   WBC     hadm_id  \
subject_id hadm_id                                             
10000032   22595853.0 1         71.0  3.80   4.2  22595853.0   
           22841357.0 3        137.0  3.60   6.6  22841357.0   
           25742920.0 8        133.0  3.36   7.5  25742920.0   
                      9        107.0  3.27   5.6  25742920.0   
           29079034.0 5         94.0  3.40   4.1  29079034.0   
...                              ...   ...   ...         ...   
19999987   23865745.0 2219875  120.0  3.64  11.6  23865745.0   
                      2219876  135.0  3.58  10.0  23865745.0   
                      2219877  141.0  3.63   5.9  23865745.0   
                      2219878  129.0  3.32   5.0  23865745.0   
                      2219879  129.0  3.52   5.7  23865745.0   

                                       jcharttime  transfer_id  ...  \
subject_id hadm_id                                              ...   
10000032   22595853.0 1       2180-05-07 05:05:00   35223874.0  ...   
           22841357.0 3       2180-06-27 05:10:00   34703856.0  ...   
           25742920.0 8       2180-08-06 06:36:00   35509340.0  ...   
                      9       2180-08-07 06:15:00   35509340.0  ...   
           29079034.0 5       2180-07-24 06:35:00   35888873.0  ...   
...                                           ...          ...  ...   
19999987   23865745.0 2219875 2145-11-04 10:40:00   36195440.0  ...   
                      2219876 2145-11-05 06:10:00   30249304.0  ...   
                      2219877 2145-11-06 10:07:00   30249304.0  ...   
                      2219878 2145-11-07 06:00:00   30249304.0  ...   
                      2219879 2145-11-09 05:30:00   30249304.0  ...   

                                          outtime                  Icu  Type  \
subject_id hadm_id                                                             
10000032   22595853.0 1       2180-05-07 17:21:27                  NaN   GEN   
           22841357.0 3       2180-06-27 18:49:12                  NaN   GEN   
           25742920.0 8       2180-08-07 17:50:44                  NaN   GEN   
                      9       2180-08-07 17:50:44                  NaN   GEN   
           29079034.0 5       2180-07-24 19:52:58                  NaN   GEN   
...                                           ...                  ...   ...   
19999987   23865745.0 2219875 2145-11-04 21:29:30  Trauma SICU (TSICU)  SICU   
                      2219876 2145-11-11 13:00:47                  NaN   GEN   
                      2219877 2145-11-11 13:00:47                  NaN   GEN   
                      2219878 2145-11-11 13:00:47                  NaN   GEN   
                      2219879 2145-11-11 13:00:47                  NaN   GEN   

                                          jintime            jouttime  \
subject_id hadm_id                                                      
10000032   22595853.0 1       2180-05-06 23:30

In [219]:
lab_patients_diag_events[lab_patients_diag_events["hadm_id"] ==21679796]

,subject_id,charttime,HGB,MCV,PLT,RBC,WBC,hadm_id,jcharttime,transfer_id,...,outtime,Icu,Type,jintime,jouttime,subject_id_charttime,gender,Age,Diagnosis,isNewWard
544998,12468016,2127-08-23 23:33:00,6.2721,100.0,324.0,2.99,17.0,21679796.0,2127-08-23 23:33:00,32033779.0,...,2127-08-25 14:41:31,Medical/Surgical Intensive Care Unit (MICU/SICU),CICU,2127-08-23 20:48:00,2127-08-25 14:41:31,124680162127-08-23 23:33:00.000,M,50,Sepsis,False
544999,12468016,2127-08-24 05:34:00,6.2100,99.0,325.0,2.96,13.9,21679796.0,2127-08-24 05:34:00,32033779.0,...,2127-08-25 14:41:31,Medical/Surgical Intensive Care Unit (MICU/SICU),CICU,2127-08-23 20:48:00,2127-08-25 14:41:31,124680162127-08-24 05:34:00.000,M,50,Sepsis,False
545000,12468016,2127-08-25 05:26:00,6.3342,100.0,384.0,2.99,14.3,21679796.0,2127-08-25 05:26:00,32033779.0,...,2127-08-25 14:41:31,Medical/Surgical Intensive Care Unit (MICU/SICU),CICU,2127-08-23 20:48:00,2127-08-25 14:41:31,124680162127-08-25 05:26:00.000,M,50,Sepsis,False
545001,12468016,2127-08-26 05:48:00,5.8995,102.0,435.0,2.86,11.1,21679796.0,2127-08-26 05:48:00,39517483.0,...,2127-08-29 18:09:42,NaN,GEN,2127-08-25 14:41:31,2127-08-29 18:09:42,124680162127-08-26 05:48:00.000,M,50,Sepsis,False
545002,12468016,2127-08-27 05:34:00,6.2721,105.0,515.0,3.00,7.8,21679796.0,2127-08-27 05:34:00,39517483.0,...,2127-08-29 18:09:42,NaN,GEN,2127-08-25 14:41:31,2127-08-29 18:09:42,124680162127-08-27 05:34:00.000,M,50,Sepsis,False
545003,12468016,2127-08-28 05:02:00,5.6511,103.0,467.0,2.75,5.7,21679796.0,2127-08-28 05:02:00,39517483.0,...,2127-08-29 18:09:42,NaN,GEN,2127-08-25 14:41:31,2127-08-29 18:09:42,124680162127-08-28 05:02:00.000,M,50,Sepsis,False


In [18]:
import time
for i in range(100):
    time.sleep(10)
    print("Hello")

Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello


KeyboardInterrupt: 